In [ ]:
!pip install anvil-uplink


  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [ ]:
import pandas as pd
dataset=pd.read_csv("/content/drive/MyDrive/Twitter_dataSet/labeled_data.csv")
dataset=dataset["tweet"]
import nltk
nltk.download('punkt_tab')
import nltk
nltk.download('stopwords')
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

filtered=[]

for i in dataset:
    text = i
    text = re.sub(r'[^\w\s]', '', text.lower())
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    filtered.append(filtered_tokens)
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch


model_name = "Hate-speech-CNERG/dehatebert-mono-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


def predict_hate_speech(text):

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)


    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    softmax = torch.nn.Softmax(dim=1)
    probs = softmax(logits)

    hate_speech_prob = probs[0][1].item()

    if hate_speech_prob > 0.5:
        return "Hate Speech", hate_speech_prob
    else:
        return "Non-Hate Speech", hate_speech_prob

toxicity={}
for tweets in filtered[:50]:
    for tweet in tweets:
        label, probability = predict_hate_speech(tweet)
        toxicity[tweet]=label

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
toxicity

{'rt': 'Non-Hate Speech',
 'mayasolovely': 'Non-Hate Speech',
 'woman': 'Non-Hate Speech',
 'shouldnt': 'Non-Hate Speech',
 'complain': 'Non-Hate Speech',
 'cleaning': 'Non-Hate Speech',
 'house': 'Non-Hate Speech',
 'amp': 'Non-Hate Speech',
 'man': 'Non-Hate Speech',
 'always': 'Non-Hate Speech',
 'take': 'Non-Hate Speech',
 'trash': 'Non-Hate Speech',
 'mleew17': 'Non-Hate Speech',
 'boy': 'Non-Hate Speech',
 'dats': 'Non-Hate Speech',
 'coldtyga': 'Non-Hate Speech',
 'dwn': 'Non-Hate Speech',
 'bad': 'Hate Speech',
 'cuffin': 'Non-Hate Speech',
 'dat': 'Non-Hate Speech',
 'hoe': 'Hate Speech',
 '1st': 'Non-Hate Speech',
 'place': 'Non-Hate Speech',
 'urkindofbrand': 'Non-Hate Speech',
 'dawg': 'Non-Hate Speech',
 '80sbaby4life': 'Non-Hate Speech',
 'ever': 'Non-Hate Speech',
 'fuck': 'Non-Hate Speech',
 'bitch': 'Hate Speech',
 'start': 'Non-Hate Speech',
 'cry': 'Non-Hate Speech',
 'confused': 'Non-Hate Speech',
 'shit': 'Non-Hate Speech',
 'c_g_anderson': 'Non-Hate Speech',
 'viv

In [ ]:
import anvil.server

anvil.server.connect("server_VRABEAOTS6DZLVDUG3DSLBNZ-5TDTS4DZ3LZMRXAU")

In [ ]:
!pip install pytesseract

In [ ]:
# final code

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import base64
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import anvil.server

# Load the hate speech model and tokenizer
model_name = "Hate-speech-CNERG/dehatebert-mono-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def predict_hate_speech(text):
    """
    Function to predict whether a sentence is hate speech or not.
    """
    # Tokenize the input sentence
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Ensure inputs are in dictionary format with 'input_ids' and 'attention_mask'
    assert 'input_ids' in inputs and 'attention_mask' in inputs, \
        "Input should be a dictionary containing 'input_ids' and 'attention_mask'"

    # Predict with the model
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    softmax = torch.nn.Softmax(dim=1)
    probs = softmax(logits)

    # Get probabilities for both classes (non-hate and hate)
    hate_speech_prob = probs[0][1].item()  # Probability of hate speech
    non_hate_prob = probs[0][0].item()  # Probability of non-hate speech

    return non_hate_prob, hate_speech_prob

def create_detailed_explanation(sentence, hate_prob, non_hate_prob):
    """
    Generates a detailed explanation for why the sentence is classified as hate or non-hate speech.
    """
    explanation = f"### Analysis of the Sentence: '{sentence}'\n\n"

    # Explain classification
    if hate_prob > non_hate_prob:
        explanation += f"**Classification**: The sentence is classified as **Hate Speech** because the model predicts a higher probability for hate speech ({hate_prob*100:.2f}%) compared to non-hate speech ({non_hate_prob*100:.2f}%).\n\n"
    else:
        explanation += f"**Classification**: The sentence is classified as **Non-Hate Speech** because the model predicts a higher probability for non-hate speech ({non_hate_prob*100:.2f}%) compared to hate speech ({hate_prob*100:.2f}%).\n\n"

    # Explain token contributions (simulated reasoning for now)
    explanation += "**Key Observations:**\n"
    explanation += "- The model analyzed the sentence at a token level to determine the context and tone of the sentence.\n"
    explanation += "- Words associated with aggression, hate, or negativity were given higher importance by the model during classification.\n"

    # Tone and contextual reasoning
    if hate_prob > non_hate_prob:
        explanation += "- The model detected potential signs of aggression or hostility in the language used.\n"
        explanation += "- The overall tone of the sentence, as interpreted by the model, indicates harmful or offensive intent.\n"
    else:
        explanation += "- The model found no significant indicators of hate speech in the sentence.\n"
        explanation += "- The language used is either neutral or lacks aggressive tones, which led the model to classify the sentence as non-hate speech.\n"

    # Confidence Insights
    explanation += f"\n**Confidence Levels:**\n"
    explanation += f"- Probability of Non-Hate Speech: {non_hate_prob*100:.2f}%\n"
    explanation += f"- Probability of Hate Speech: {hate_prob*100:.2f}%\n"

    return explanation

def generate_image(sentence, hate_prob, non_hate_prob):
    """
    Generates a bar chart representing the probabilities of hate and non-hate speech.
    """
    fig, ax = plt.subplots(figsize=(8, 5))
    categories = ["Non-Hate", "Hate"]
    probabilities = [non_hate_prob, hate_prob]
    colors = ['#8A2BE2', '#FF4500']  # Custom colors

    bars = plt.bar(categories, probabilities, color=colors, edgecolor='black', linewidth=1.5)

    # Add sentence above the histogram
    plt.text(0.5, 1.05, f"Sentence: {sentence}", ha='center', va='center', transform=ax.transAxes,
             fontsize=12, fontweight='bold', bbox=dict(facecolor='white', alpha=0.8, edgecolor='black'))

    # Add annotations
    for bar, prob in zip(bars, probabilities):
        ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.02,
                f"{prob:.3f}", ha='center', fontsize=12, fontweight='bold')

    # Save the plot to an in-memory image
    image_stream = BytesIO()
    plt.savefig(image_stream, format="png")
    image_stream.seek(0)

    # Encode image in base64
    encoded_image = base64.b64encode(image_stream.read()).decode('utf-8')
    return encoded_image

def analyze_sentence(sentence):
    """
    Main function to analyze the sentence and return hate speech probabilities, an image, and explanation.
    """
    # Get probabilities of non-hate and hate speech
    non_hate_prob, hate_prob = predict_hate_speech(sentence)

    # Create a detailed explanation based on the probabilities
    explanation = create_detailed_explanation(sentence, hate_prob, non_hate_prob)

    # Generate a bar chart image for visualization
    encoded_image = generate_image(sentence, hate_prob, non_hate_prob)

    return non_hate_prob, hate_prob, encoded_image, explanation

# Callable function for Anvil
@anvil.server.callable
def get_analysis(sentence):
    """
    Callable function that is used to get the analysis of a sentence
    when called from the Anvil front-end.
    """
    return analyze_sentence(sentence)

# Keep the Anvil server running
anvil.server.wait_forever()


<ipython-input-6-156a9d1beb70>:93: UserWarning: Glyph 9 (	) missing from current font.
  plt.savefig(image_stream, format="png")


In [ ]:
# import matplotlib.pyplot as plt
# from matplotlib.patches import Rectangle
# import torch
# from transformers import AutoModelForSequenceClassification, AutoTokenizer
# from PIL import Image
# import io

# # Function to generate the hate speech histogram and save it as an image
# def generate_hate_speech_histogram(sentence):
#     # Generate the hate speech probabilities
#     model_name = "Hate-speech-CNERG/dehatebert-mono-english"
#     model = AutoModelForSequenceClassification.from_pretrained(model_name)
#     tokenizer = AutoTokenizer.from_pretrained(model_name)

#     inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512)
#     with torch.no_grad():
#         outputs = model(**inputs)
#     logits = outputs.logits
#     softmax = torch.nn.Softmax(dim=1)
#     probs = softmax(logits)

#     hate_speech_prob = probs[0][1].item()  # Probability of hate speech
#     non_hate_prob = probs[0][0].item()  # Probability of non-hate speech

#     # Create a histogram plot
#     fig, ax = plt.subplots(figsize=(8, 5))
#     categories = ["Non-Hate", "Hate"]
#     probabilities = [non_hate_prob, hate_speech_prob]
#     colors = ['#8A2BE2', '#FF4500']  # Custom colors

#     bars = plt.bar(categories, probabilities, color=colors, edgecolor='black', linewidth=1.5)

#     # Add sentence above the histogram
#     plt.text(0.5, 1.05, f"Sentence: {sentence}", ha='center', va='center', transform=ax.transAxes,
#              fontsize=12, fontweight='bold', bbox=dict(facecolor='white', alpha=0.8, edgecolor='black'))

#     # Add annotations
#     for bar, prob in zip(bars, probabilities):
#         ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.02,
#                 f"{prob:.3f}", ha='center', fontsize=12, fontweight='bold')

#     # Set the background color and other aesthetic improvements
#     ax.set_facecolor('#F0F8FF')
#     ax.spines['top'].set_visible(False)
#     ax.spines['right'].set_visible(False)
#     ax.spines['left'].set_linewidth(1.5)
#     ax.spines['bottom'].set_linewidth(1.5)
#     plt.grid(axis='y', linestyle='--', alpha=0.7)
#     plt.xlabel("Category", fontsize=14)
#     plt.ylabel("Probability", fontsize=14)
#     plt.ylim(0, 1)

#     # Save the figure as an image in memory (using BytesIO)
#     img_stream = io.BytesIO()
#     plt.savefig(img_stream, format='png')
#     img_stream.seek(0)
#     img = Image.open(img_stream)

#     # Save the image file and return it (this will be used in Anvil)
#     img.save("/content/hate_speech_histogram.png")

#     return "/content/hate_speech_histogram.png", non_hate_prob, hate_speech_prob, sentence

# # Example usage in Colab
# sentence = "I love programming, but hate when things go wrong!"
# image_path, non_hate_prob, hate_prob, text = generate_hate_speech_histogram(sentence)
